In [24]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [25]:
# Cargar los archivos
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [26]:
# Mostrar las primeras filas de ambos archivos para entender su estructura
train_data.head()


,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,697,705,Asus,Chromebook Flip,2 in 1 Convertible,12.5,Full HD / Touchscreen 1920x1080,Intel Core M M3-6Y30 0.9GHz,4GB,64GB Flash Storage,Intel HD Graphics 515,Chrome OS,1.2kg,669.0
1,435,442,Asus,Rog Strix,Gaming,17.3,Full HD 1920x1080,AMD Ryzen 1600 3.2GHz,8GB,256GB SSD + 1TB HDD,AMD Radeon RX 580,Windows 10,3.2kg,1695.0
2,735,743,Lenovo,V310-15IKB (i7-7500U/4GB/1TB/FHD/W10),Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,4GB,1TB HDD,Intel HD Graphics 620,Windows 10,1.85kg,779.0
3,864,875,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7660U 2.5GHz,16GB,512GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.29kg,2240.0
4,1176,1194,Lenovo,B51-80 (i7-6500U/4GB/1008GB/FHD/W7),Notebook,15.6,Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,1.0TB Hybrid,Intel HD Graphics 520,Windows 7,2.32kg,825.0


In [27]:
test_data.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,824,833,Lenovo,ThinkPad X1,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB Flash Storage,Intel HD Graphics 620,Windows 10,1.13kg
1,101,104,HP,15-bw000nv (E2-9000e/4GB/500GB/Radeon,Notebook,15.6,Full HD 1920x1080,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,Windows 10,2.1kg
2,1157,1175,Lenovo,ThinkPad T460s,Ultrabook,14.0,Full HD 1920x1080,Intel Core i7 6600U 2.6GHz,12GB,512GB SSD,Intel HD Graphics 520,Windows 7,1.4kg
3,940,954,HP,Omen 17-W006na,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 6300HQ 2.3GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 965M,Windows 10,2.75kg
4,853,864,Lenovo,ThinkPad T470p,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia GeForce GT 940MX,Windows 10,1.96kg


In [28]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [29]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                391 non-null    int64  
 1   laptop_ID         391 non-null    int64  
 2   Company           391 non-null    object 
 3   Product           391 non-null    object 
 4   TypeName          391 non-null    object 
 5   Inches            391 non-null    float64
 6   ScreenResolution  391 non-null    object 
 7   Cpu               391 non-null    object 
 8   Ram               391 non-null    object 
 9   Memory            391 non-null    object 
 10  Gpu               391 non-null    object 
 11  OpSys             391 non-null    object 
 12  Weight            391 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 39.8+ KB


In [30]:
# Conversión de 'Ram' a numérico
train_data['Ram'] = train_data['Ram'].str.extract('(\d+)').astype(float)

# Eliminar 'kg' de la columna 'Weight' y convertir a numérico
train_data['Weight'] = train_data['Weight'].str.replace('kg', '').astype(float)

# Convertir 'Gpu' a numérico
train_data['Gpu'] = train_data['Gpu'].apply(lambda x: 1 if 'Nvidia' in x else (2 if 'Intel' in x else 0))

# Convertir las columnas categóricas en variables numéricas usando One-Hot Encoding
X_train = pd.get_dummies(train_data, columns=['Company', 'TypeName', 'OpSys'])

# Asegurarnos de que 'ScreenResolution' esté en formato de string antes de extraer la resolución
X_train['ScreenResolution'] = X_train['ScreenResolution'].astype(str)
X_train['ScreenResolution'] = X_train['ScreenResolution'].str.extract('(\d+)').astype(float)

# Separar el target (Precio) del conjunto de características
y_train = X_train['Price_euros']
X_train = X_train.drop(columns=['Price_euros'])

# Verificar las primeras filas del conjunto de datos procesados
X_train.head()


,id,laptop_ID,Product,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,Weight,...,TypeName_Workstation,OpSys_Android,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS
0,697,705,Chromebook Flip,12.5,1920.0,Intel Core M M3-6Y30 0.9GHz,4.0,64GB Flash Storage,2,1.20,...,False,False,True,False,False,False,False,False,False,False
1,435,442,Rog Strix,17.3,1920.0,AMD Ryzen 1600 3.2GHz,8.0,256GB SSD + 1TB HDD,0,3.20,...,False,False,False,False,False,False,True,False,False,False
2,735,743,V310-15IKB (i7-7500U/4GB/1TB/FHD/W10),15.6,1920.0,Intel Core i7 7500U 2.7GHz,4.0,1TB HDD,2,1.85,...,False,False,False,False,False,False,True,False,False,False
3,864,875,XPS 13,13.3,3200.0,Intel Core i7 7660U 2.5GHz,16.0,512GB SSD,2,1.29,...,False,False,False,False,False,False,True,False,False,False
4,1176,1194,B51-80 (i7-6500U/4GB/1008GB/FHD/W7),15.6,1920.0,Intel Core i7 6500U 2.5GHz,4.0,1.0TB Hybrid,2,2.32,...,False,False,False,False,False,False,False,False,True,False


In [31]:
y_train.head()

0     669.0
1    1695.0
2     779.0
3    2240.0
4     825.0
Name: Price_euros, dtype: float64

In [32]:
# Conversión de 'Ram' a numérico
test_data['Ram'] = test_data['Ram'].str.extract('(\d+)').astype(float)

# Eliminar 'kg' de la columna 'Weight' y convertir a numérico
test_data['Weight'] = test_data['Weight'].str.replace('kg', '').astype(float)

# Convertir 'Gpu' a numérico
test_data['Gpu'] = test_data['Gpu'].apply(lambda x: 1 if 'Nvidia' in x else (2 if 'Intel' in x else 0))

# Convertir las columnas categóricas en variables numéricas usando One-Hot Encoding
X_test = pd.get_dummies(test_data, columns=['Company', 'TypeName', 'OpSys'])

# Asegurarnos de que 'ScreenResolution' esté en formato de string antes de extraer la resolución
X_test['ScreenResolution'] = X_test['ScreenResolution'].astype(str)
X_test['ScreenResolution'] = X_test['ScreenResolution'].str.extract('(\d+)').astype(float)

# Alinear las columnas entre X_train y X_test (en caso de que haya diferencias en las columnas)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Verificar las primeras filas después del preprocesamiento
X_test.head()


,id,laptop_ID,Product,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,Weight,...,TypeName_Workstation,OpSys_Android,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS
0,824,833,ThinkPad X1,14.0,1920.0,Intel Core i5 7200U 2.5GHz,8.0,256GB Flash Storage,2,1.13,...,False,0,False,False,False,False,True,False,False,False
1,101,104,15-bw000nv (E2-9000e/4GB/500GB/Radeon,15.6,1920.0,AMD E-Series E2-9000e 1.5GHz,4.0,500GB HDD,0,2.10,...,False,0,False,False,False,False,True,False,False,False
2,1157,1175,ThinkPad T460s,14.0,1920.0,Intel Core i7 6600U 2.6GHz,12.0,512GB SSD,2,1.40,...,False,0,False,False,False,False,False,False,True,False
3,940,954,Omen 17-W006na,17.3,1920.0,Intel Core i5 6300HQ 2.3GHz,8.0,128GB SSD + 1TB HDD,1,2.75,...,False,0,False,False,False,False,True,False,False,False
4,853,864,ThinkPad T470p,14.0,1920.0,Intel Core i7 7700HQ 2.8GHz,8.0,256GB SSD,1,1.96,...,False,0,False,False,False,False,True,False,False,False


In [33]:
from sklearn.ensemble import RandomForestRegressor

# Crear el modelo de Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Predecir sobre el conjunto de prueba
y_pred = model.predict(X_test)


ValueError: could not convert string to float: 'Chromebook Flip'